In [1]:
# envionment setup
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
import site
site.addsitedir('../..')

## APIs to retrieve nodes in the graph

In [2]:
# to retrieve nodes in the graph

from emr.graph import graph

# retrieve by __getattr__
print(graph.linguistic.phrase)
# retrieve by __getitem__
print(graph['linguistic/phrase'])
# retrieve by properties
print(graph.subs['linguistic'].concepts['phrase'])
# TODO: we might also add wildcard? should be like glob module does.
# Example: graph['*/phrase']

# or even play around
print(graph.subs['linguistic'].sup.linguistic.phrase)
print(graph.subs['linguistic'].sup['linguistic/phrase'])

Concept(name='phrase', what={'out_rels': {Concept(name='word'): {Have(name='(phrase)-have-(0:word)', what={'dst': {0: Concept(name='word')},
 'src': Concept(name='phrase')})}}})
Concept(name='phrase', what={'out_rels': {Concept(name='word'): {Have(name='(phrase)-have-(0:word)', what={'dst': {0: Concept(name='word')},
 'src': Concept(name='phrase')})}}})
Concept(name='phrase', what={'out_rels': {Concept(name='word'): {Have(name='(phrase)-have-(0:word)', what={'dst': {0: Concept(name='word')},
 'src': Concept(name='phrase')})}}})
Concept(name='phrase', what={'out_rels': {Concept(name='word'): {Have(name='(phrase)-have-(0:word)', what={'dst': {0: Concept(name='word')},
 'src': Concept(name='phrase')})}}})
Concept(name='phrase', what={'out_rels': {Concept(name='word'): {Have(name='(phrase)-have-(0:word)', what={'dst': {0: Concept(name='word')},
 'src': Concept(name='phrase')})}}})


## Experimental Area

## Developing Area

In [3]:
from allennlp.models.model import Model
from allennlp.data.vocabulary import Vocabulary
import torch
from typing import Dict


class BaseModel(Model):
    def __init__(self, vocab: Vocabulary) -> None:
        super().__init__(vocab)

    def _update_metrics(self, **data):
        for metric_name, metric in self.metrics.items():
            metric(data['logits'], data['labels'], data['metric_mask'])
            data[metric_name] = metric.get_metric(False) # no reset
        return data
    
    def get_metrics(self, reset: bool = False) -> Dict[str, float]:
        output = {}
        for metric_name, metric in self.metrics.items():
            output[metric_name] = metric.get_metric(reset)
        return output

    def _update_loss(self, **data):
        if self.loss_func is not None:
            data['loss'] = self.loss_func(**data)
        return data

    def forward(
        self,
        **data:Dict[str, torch.Tensor]
    ) -> Dict[str, torch.Tensor]:
        
        ##
        # something happen here to take the input to the output
        ##

        return data


Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [4]:
from typing import List, Dict, Callable
from allennlp.models.model import Model
from torch import Tensor

data_source = List[Dict[str, Tensor]]
# NB: list of Instance(dict of str to data format use in module)
#     For allen nlp, Instance is Dict of str:Field,
#     and real tensor will be there in forward function
module_func = Callable[[Dict[str, Tensor]], Dict[str, Tensor]]
# NB: modules are transform from Dict of str:Tensor to updated Dict
#     module objects in AllenNLP have the forward function of this setting

class Library(object):
    def __init__(self) -> None:
        self.modules = dict()
        self.loss = None

    def data(
        self,
        data_source: data_source
    ) -> module_func:
        # return output_prop
        pass

    def word2vec(
        self,
        input_prop: module_func
    ) -> module_func:
        # return output_prop
        pass

    def fc_sm(
        self,
        input_prop: module_func
    ) -> module_func:
        # return output_prop
        pass

    def build(
        self
    ) -> Model:
        pass


In [5]:
class Data(object):
    pass

## Baseline code

In [6]:
from emr.data import Conll04DatasetReader as Reader
from emr.model import get_trainer
#from emr.graph import graph

# types for typing style function
from typing import List, Dict
from allennlp.data.vocabulary import Vocabulary
from allennlp.data.instance import Instance
from allennlp.models.model import Model
from regr.graph import Graph
from allennlp.training.trainer import Trainer

In [7]:
from regr import Graph, Concept


Graph.clear()
Concept.clear()
with Graph('global') as graph:
    word = Concept(name='word')
    people = Concept(name='people')
    people.be(word)

graph

Graph(name='global', what={'concepts': {'people': Concept(name='people', what={'out_rels': {Concept(name='word'): {Be(name='(people)-be-(0:word)', what={'dst': {0: Concept(name='word')},
 'src': Concept(name='people')})}}}),
              'word': Concept(name='word', what={'out_rels': {}})},
 'subs': {},
 'sup': None})

In [8]:
from emr.graph import Concept

from allennlp.data.vocabulary import Vocabulary

# develop by an ML programmer to wire nodes in the graph and ML Models
def make_model(graph: Graph,
               data: Data, # oh what is this?
               lib: Library, # this is not Model we have
              ) -> Model:
    # get concepts from graph
    word = graph.word
    people = graph.people
    
    # binding
    # filling in data and label
    word['index'] = lib.data(data.input) # (batch, len)
    people['label'] = lib.data(data.peop_label) # (batch, len,)
    
    # building model
    word['w2v'] = lib.word2vev(word['index']) # (batch, len, vec)
    people['label'] = lib.fc_sm(word['w2v']) # (batch, len,)
    # now people['label'] has multiple assignment,
    # and the loss should come from the inconsistency here

    # get the model
    model = lib.build()
    # NB: Link in the graph make be use to provide non parameterized
    #     transformation, what is a core feature of our graph.
    #     Is there a better semantic interface design?
    
    # get the loss
    model.loss = graph()
    
    return model

In [9]:
# data setting
relative_path = "data/EntityMentionRelation"
train_path = "conll04_train.corp"
valid_path = "conll04_test.corp"

# model setting
EMBEDDING_DIM = 16
HIDDEN_DIM = 8

# training setting
LR = 0.1
BATCH = 16
EPOCH = 10 # 1000
PATIENCE = 10

In [10]:
# envionment setup

#import logging
# logging.basicConfig(level=logging.INFO)

def seed1():
    import random
    import numpy as np
    import torch
    
    np.random.seed(1)
    random.seed(1)
    torch.manual_seed(1)
    
seed1()

In [11]:
import os
from allennlp.data.vocabulary import Vocabulary

# data
reader = Reader()
train_dataset = reader.read(os.path.join(relative_path, train_path))
valid_dataset = reader.read(os.path.join(relative_path, valid_path))
vocab = Vocabulary.from_instances(train_dataset + valid_dataset)
data = None # TODO: what is data now

# model from graph
lib = Library()
model = make_model(graph, data, lib)

# trainer for model
trainer = get_trainer(model, train_dataset, valid_dataset)

# train the model
trainer.train()

# save the model
with open("/tmp/model.th", 'wb') as fout:
    torch.save(model.state_dict(), fout)
vocab.save_to_files("/tmp/vocab")

0it [00:00, ?it/s]
 96%|█████████▌| 69261/72519 [00:00<00:00, 192888.90it/s]
2942it [00:00, 4665.27it/s]9 [00:00<00:00, 195975.51it/s]
0it [00:00, ?it/s]
 71%|███████   | 49433/69723 [00:00<00:00, 244447.86it/s]
2573it [00:00, 4825.49it/s]3 [00:00<00:00, 245670.85it/s]
100%|██████████| 5515/5515 [00:00<00:00, 44250.65it/s]


AttributeError: 'NoneType' object has no attribute 'input'